<a href="https://colab.research.google.com/github/lethiciaasevedo1999/PySpark/blob/main/PySpark_Alura.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Instalando dependências**

In [1]:
!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


# **Imports**

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [32]:
import findspark
findspark.init()

# **Iniciando sessão Spark**

In [33]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Iniciando com Spark") \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

# **Download das extensões Ngrok**

In [34]:
!wget -q https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   


**Instalando pyngrok para inserir o token de acesso**

In [21]:
!pip install pyngrok

**Inserindo token para validar a sessão**

In [42]:
from pyngrok import ngrok

ngrok.set_auth_token("2ou3jc47yccpmRLaifd195JYbUr_7fe294WYvXrx9n9CqZhR4")

In [43]:

get_ipython().system_raw('./ngrok http 4050 &')

**Gerando link de acesso**

In [44]:
!curl -s http://localhost:4040/api/tunnels

{"tunnels":[{"name":"http-4050-990dc64c-4e18-4746-85f7-fae71f6e2ca0","ID":"e4cc4aed27c68679ca27a9f55c99f791","uri":"/api/tunnels/http-4050-990dc64c-4e18-4746-85f7-fae71f6e2ca0","public_url":"https://299a-35-227-166-115.ngrok-free.app","proto":"https","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":73,"gauge":0,"rate1":0.00016196865239689285,"rate5":0.040614590504445144,"rate15":0.04464530209569474,"p50":334721669,"p90":1964468292.0000021,"p95":13598552753.799967,"p99":75405986232},"http":{"count":94,"rate1":0.00020202635100229508,"rate5":0.05202774463553124,"rate15":0.05739206274750231,"p50":10514424,"p90":218807375,"p95":466555205.25,"p99":2244759074}}}],"uri":"/api/tunnels"}


# **Criando o dataframe**

In [39]:
data = [('Zeca', '35'), ('Eva', '29')]
colNames = ['Nome', 'Idade']
df = spark.createDataFrame(data, colNames)

df.show()

+----+-----+
|Nome|Idade|
+----+-----+
|Zeca|   35|
| Eva|   29|
+----+-----+



**Comparando diferença entre dataframe pandas e pyspark**

In [40]:
df.toPandas()

,Nome,Idade
0,Zeca,35
1,Eva,29
